## For Japanese rookie by Japanese rookie
# **Jane Street: EDA of day 0 and feature importance**

 This notebook is begginers version of 
[Jane Street: EDA of day 0 and feature importance](https://www.kaggle.com/carlmcbrideellis/jane-street-eda-of-day-0-and-feature-importance) made by [Mr.Carl McBride Ellis](https://www.kaggle.com/carlmcbrideellis).

I respect him, and I'm impressed by this notebook!  
But I'm a beginner,and too new to understand this instantly.  
So, I add supplements for ***beginners*** and Japanese translation.  
I made this just for my own growth.  
If you are a beginner who is confused about understanding his notebook(Same trouble as me), I may be able to help you (especially Japanese).   
### <span style="color: red; ">*Note:If there are copyright issues ,offensive authors, or anything else,this notebook will be deleted immediately.*</span>
Please forgive me though it is a childish sentence and English.  



### ●Description
「低く買って高く売る」は簡単に見えて、難しい  
電子取引によりリアルタイムで価格差を見つけ利用できる可能性が無限大  
全員が合理的とすると、価格はいつも公平な価格になるが、現実はそうではない  
非効率性を利用する取引戦略を作るのは難しい（良い意思決定orただの幸運）  
　①膨大な取引データからリターンを最大化する定量的なモデルを作る  
　②将来の市場returnに対してモデルによる予測をする　　
   
historical dataを、モデルを作るために数学的手法と技術的手法を用いる  　　  
　potential trading opportunitiesの数が与えられる　　  
良いモデルができ正しく取引を実行するとより公正な価格になるが、現実的には厳しい　　  
JaneStreetはtrading modelsとmachine learning solutionsを作った　　  
　…収益性の高いopportunitiesを特定し、取引の実行を迅速に決定する　　  
JSの量的問題をはるかに単純化してるが、この課題の解決の新しいアプローチを求めている  　　
　　
### ●Evaluation：a utility score　　　  
test setのrow：action value をpredictするa trading opportunityを表す　　  
submission file：python time-series APIを用いる　　  
　modelが時間通りにpeek forwardしないようになる（？）　　  
  
### ●Data Description　　    
feature_{0...129}：株市場のデータ　　

row(列):取引機会
action value：1なら取引、0ならスルー。これを予測する　　  
resp:収益率＝投資元本の増加分/投資元本。  
resp1~4はtime horizon(時間の区切ろい)を変えただけ。モデルの精度を上げるために使う。  
test setにはrespのみ。
weight:取引量。weight=0はscoreの評価に寄与しないが、空白を作らないために意図的に入れてある。高いweightは低いrespのみに関係。  
 
column(行)  
date:取引した日にち  
ts_id:時間の順序  


0. import：numpy,pandas,plottings,dabl,dabl,misc,system

1. The train.csv file is big：訓練用データサイズが大きい

2. resp(once)：時間の経過に伴う応答の累積値culmative valuesをみる

3. weight：tradeにはweightとrespが重要

4. Cumulative return：累積daily return＝weight * calue of resp

5. Time：各日の、ts_idの数をplots ※垂直破線：85日目付近のモデルの修正？

6. The features

7. The features.csv file：匿名化されたfeaturesに関連するメタデータ、tagは29個

8. Action(1:取引する(resp=+), 0:取引しない(resp=-)、のbinary(2進)column)

9. The first day ("day 0")：missing dataにはpattternがある

10. Are there any missing values?

11. Is there any missing data: Days 2 and 294

12. DABL plots (targets: action and resp)

13. Permutation(順序) Importance using the Random Forest

14. Is there any correlation between day 100 and day 200?

15. The test data

16. Evaluation：utility score

## 0,import：numpy,pandas,plottings,dabl,misc,system


In [ ]:
#numpy
import numpy as np

#pandas stuff
import pandas as pd
#pandasの設定値を変更（第1因数に正規表現パターンの文字列、第2因数に設定する値)
#display.max_rowsの値をNoneにする
pd.set_option('display.max_rows', None)
#display.max_columnsの値をNoneにする
pd.set_option('display.max_columns', None)

#plottings stuff
from pandas.plotting import lag_plot
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
#sns(seaborn)の定性的なカラーパレット
#as_camp=Trueならば、matplotlib.colors.Colormapを返す
colorMap = sns.light_palette("blue", as_cmap=True)


#install dabl（→chapter12）
#pipはPythonパッケージのインストールなどを行うユーティリティ（標準装備）
#> : 欲しいfileに標準出力でリダイレクトする
#　→dev/nullにリダイレクトする
#　→一種のvirtual fileとなり、そこに置いたものが削除される
#　→ディスク領域を使用しない（守るためのコマンド）
#　ただし、dev/nullはLinux用なので、nullのみでよい

#!pip install dabl > /dev/null
!pip install dabl > null
import dabl
#install datatable
!pip install datatable > /dev/null
import datatable as dt

#misc
#欠損値を効果的に可視化する
import missingno as msno

#system
#警告メッセージは、警告した方が良いときにされるが、別にプログラムを終了させるほどではない
#（バージョンが古い、など）
#warnings：警告を管理
import warnings
#実行時の紛らわしい警告を非表示(なぜかpipのwarningは消えない)
warnings.filterwarnings('ignore')

#for the image import
import os
#ipython.display：音声や動画などをNotebook内に埋め込む
#Image：画像を表示
from IPython.display import Image

#garbage collector to keep RAM in check
#RAMを確認するために不要領域を自動的に解法する機能
import gc
#この後のgc.collect()は、
#使い終わって必要なくなった（回収可能な）オブジェクト（メモリ領域）を自動的に削減する
#多分上級者向け
#コードで使用されていたメモリーがOSに返されるという保証はない
#あくまで、使用済みメモリーが将来別のオブジェクトで使用するために開放される、ことのみを保証
#どうしても、「開放されるはずのメモリが開放されない」時に使う


## 1,The train.csv file is big：訓練用データサイズが大きい
→サイズ大で時間がかかるので、datatableを使い、pandas dataframeに変換



In [ ]:
#wc -1 (ファイルパス)：データ数（行数）をカウントするコマンド
!wc -l ../input/jane-street-market-prediction/train.csv
#サイズが大きいことがわかる

In [ ]:
%%time
# 「%time」：一行の実行時間
# 「%%time」：セル一つの実行時間 　※このコマンドはセルの一番上に書かないとエラーに

#freadは読み込みが非常に速いため、data.tableとして読み込んで、その後にdataframeに変換
train_data_datatable = dt.fread('../input/jane-street-market-prediction/train.csv')

In [ ]:
%%time
#pandas datatableに変換

train_data = train_data_datatable.to_pandas()

## 2,resp(once)：時間の経過に伴う応答の累積値culmative valuesをみる
THが長い→投資家は積極的でリスク高い　TH短い→保守的でリスク低い  
※TH；TimeHorizon：（株や証券などを保有している）時間の範囲  



In [ ]:
#train.data：500days(2years of trading data)
#culmative values of resp over time

#plt : matplotlib.pyplot
#図のサイズを決める(横:縦=15:5)
#plt.figure():Figureインスタンスを作成(描画できる領域を確保)
#((横,縦),ドット数,...)
#fig.add_subplot():Figure上にグラフを描画
#plt.subplot():現在の描画領域(fig)に追加するメソッド

#plt.subplots(subplotの行数(分割数),列数,...figsize):
#fig:figure(1つのFigureクラスのインスタンス)
#ax:axes(1つのAxesSubplotクラスのインスタンス)
fig, ax = plt.subplots(figsize=(15,5))

#2次元のデータフレームに対して、Seriesは一次元配列
#cumsum:cumulative sum（累積値）
balance = pd.Series(train_data['resp']).cumsum()
##なぜbalanceという名前？

#x,yのラベル
#ax:AxesSubplotクラスのインスタンス
ax.set_xlabel("Trade", fontsize=18)
ax.set_ylabel("Cumulative resp", fontsize=18);

#balanceをplot
#lw:line width(線幅)
balance.plot(lw=3);

#balanceの変数を消してメモリを確保
del balance
gc.collect();

In [ ]:
#同様に、4time horioznsも行う

#train.csvのカラム内容
#date
#weight
#resp_1...4,resp
#feature_0...129
#ts_id

#図のサイズを決める(横:縦=15:5)
fig, ax = plt.subplots(figsize=(15,5))
#各respをのデータを変数に格納
#（train_data内の[]の線形データの累積値）
balance = pd.Series(train_data['resp']).cumsum()
resp_1 = pd.Series(train_data['resp_1']).cumsum()
resp_2 = pd.Series(train_data['resp_2']).cumsum()
resp_3 = pd.Series(train_data['resp_3']).cumsum()
resp_4 = pd.Series(train_data['resp_4']).cumsum()

#xのラベル
ax.set_xlabel("Trade", fontsize=18)
#title
ax.set_title("Cumulative resp and time horizons 1,2,3, and 4(500days)", fontsize=18);
#balance,resp_1,2,3,4をplot
balance.plot(lw=3)
resp_1.plot(lw=3)
resp_2.plot(lw=3)
resp_3.plot(lw=3)
resp_4.plot(lw=3)

#凡例
#凡例枠は左上(upper left)
plt.legend(loc="upper left");

#各変数を消してメモリ削減
del balance
del resp_1
del resp_2
del resp_3
del resp_4
gc.collect();

resp（青）はresp4(紫)に近い。

"Jane Street: time horizons and volatilities"  
→最尤推定より、time horizon(Tj)にたいし、resp_1をT1(5trading data)とすると、
* Tj(resp_2) = 1.4T1
* Tj(resp_3) = 3.9T1
* Tj(resp_4) = 11.1T1


In [ ]:
#resp値-0.05~0.05をplot：裾野が広い→全体の範囲は-0.55～0.45  
#歪度skew    ：0.10  ;分布が正規分布からどれだけ歪んでいるか（どれだけ非対称か）を表す
#尖度kurtosis：17.36 ;分布が正規分布からどれだけ尖っているか（どれだけ平均周辺に周通するか）を表す   

#図のサイズを決める(横:縦=12:5)
plt.figure(figsize=(12,5))
#sns.distplot:ヒストグラム
#(x, bins;階級幅, hist=True, 
# kde;密度近似関数の描画=True,(デフォルト)
# rug;実数地の描画=False, fit;正規分布の描画=None,
# hist_kws=None, kde_kws=None, rug_kws=None, fit_kws=None, color=None, 
# vertical=False, norm_hist=False, axlabel=None, label=None, ax=None)

##_kws：何の略？ 辞書型で、グラフに対して色やラベルを指定

ax = sns.distplot(train_data['resp'],
                bins=3000,
                kde_kws={"clip":(-0.05,0.05)},
                hist_kws={"range":(-0.05,0.05)},
                color='darkcyan', #ダークシアン(青緑色)
                kde=False);

#valuesを設定
#np.array:n次元配列
#matplotlib.patches：円、楕円、長方形など様々な図形を描画するクラスが用意されている
#上のヒストグラムの各データにおける高さを取得しvaluesに格納
values = np.array([rec.get_height() for rec in ax.patches])

#matplotlibのcolorbarは、0～1の間の値に対応して色が定義されている
#今回はvalues.min～values.maxの間の値に対応して色を定義したい
#なので、valuesのminとmaxを用いて標準化
norm = plt.Normalize(values.min(), values.max())
#plt.cm:グラデーション的に変える(.jet:可視化方法の一つ、hsv,spectralなどもある)
colors = plt.cm.jet(norm(values))

#zip:複数のobjectの要素を同時に取得
#つまり、ax.pathcesとcolorsの値を対応させたうえで、recとcolに格納する
##recvfrom:戻り値、column:カラム?
for rec, col in zip(ax.patches, colors):
    #rec(ヒストグラムデータ)に色を付ける
    rec.set_color(col)

#xラベルを加えて表示
plt.xlabel("Histogram of the resp values", size=14)
plt.show();
gc.collect();

In [ ]:
#respの,min,max,skew,kurtosisを計算
min_resp = train_data["resp"].min()
max_resp = train_data["resp"].max()

#.formatの.5fは、小数点以下第5位まで表示

#print('The minimum value for resp is: %.5f' % min_resp)
print('The minimum value for resp is: {:.5f}' .format(min_resp))

#print('The maximum value for resp is: %.5f' % max_resp)
print('The maximum value for resp is: {:.5f}'.format(max_resp))

#print("Skew of resp is: %.2f" % train_data['resp'].skew())
#print("Kurtosis of resp is: %.2f" %train_data['resp'].kurtosis())
print("Skew of resp is: {:.2f}" .format(train_data['resp'].skew()))
print("Kurtosis of resp is: {:.2f}" .format(train_data['resp'].kurtosis()))


In [ ]:
#Cauchy distribution；コーシー分布
#連続確率分布の一種で、左右対称で、期待値は存在しない（外れ値をとる確率が高い（裾が重い））
#大数の法則が成り立たず、最頻値＝中央値は定義される

#scipy>optimize>curve_fit(カーブフィッチング;曲線近似)
from scipy.optimize import curve_fit

#the values
#values:ヒストグラムの各データにおける高さを格納したもの
#それの要素数をリストに格納;x=[0,1,...]
x = list(range(len(values)))
#xの値 = (x-1500)/30000
x = [((i)-1500)/30000 for i in x]
y = values

#ローレンツ関数を定義(コーシー分布＝ローレンツ分布)
def Lorentzian(x, x0, gamma, A):
    return A * gamma**2 / (gamma**2+(x-x0)**2)

#seed guess
#種を予測する
#初めの値を設定
initial_guess = (0,0.001, 3000)

#the fit
#ローレンツ関数に曲線近似させ、パラメータと共分散を求める
parameters,covariance = curve_fit(Lorentzian, x, y, initial_guess)
#共分散の体格成分を抽出し、平方根をとる(=sigma)
sigma = np.sqrt(np.diag(covariance))

#and plot
plt.figure(figsize = (12,5))
#distplot:ヒストグラム
ax = sns.distplot(train_data['resp'],
                 bins = 3000,
                 kde_kws = {"clip":(-0.05,0.05)},
                 hist_kws = {"range":(-0.05,0.05)},
                 color = 'darkcyan',
                 kde = False);

values = np.array([rec.get_height() for rec in ax.patches])
#上で同じことやったので省略
#norm = plt.Normalize(values.min(), values.max())
#colors = plt.cm.jet(norm(values))
#for rec, col in zip(ax.patches, colors):
#    rec.set_color(col)
plt.xlabel("Histogram of the resp values", size=14)
#引数に1つの*をつけると複数の引数をタプルとして受け取る
#引数に2つの*をつけると複数のキーワード引数を辞書として受け取る
plt.plot(x, Lorentzian(x,*parameters), '--', color='black', lw=3)
plt.show();
del values
gc.collect();

Cauthy distribution は2つの独立した平均０の標準分布に従うランダムの値からなる
The paper by David E. Harris "The Distribution of Returns"   
より詳しいCauthy distibutionの説明


## 3,weight：tradeにはweightとrespが重要
意図的にweight=0も含む


In [ ]:
#weight_0の全体における割合の計算
#.shape:次元ごとの大きさを出力 
#train_data.shape
#(2390491, 138)

#(100；確率を%にするため) / (train；) * (traindataのweightが0)
#.shape[0]:一次元目(行)の合計数
#train_data > weight > valuesが0 をTrueとしたときの、Trueの合計数
percent_zeros = (100/train_data.shape[0]) * ((train_data.weight.values == 0).sum())

#print('percentage of zero weights is: %i' % percent_zeros + "%")
print('percentage of zero weights is: {:.0f}'.format(percent_zeros) + "%")

In [ ]:
#negative weightがあるかどうかをチェック（negative weightは意味ないから）
#weightのmin
min_weight = train_data['weight'].min()
#print('The minimum weight is: %.2f' % min_weight)
print('The minimum weight is: {:.2f}'.format(min_weight))


#weightのmin
max_weight = train_data['weight'].max()
#print('The maximum weight is: %.2f' % max_weight)
print('The maximum weight is: {:.2f}'.format(max_weight))


In [ ]:
#[]の中に条件を書く(weightがmax)
train_data[train_data['weight']==train_data['weight'].max()]
#446日目に起こる

In [ ]:
#weightをplot
#まずは単色でplot
plt.figure(figsize = (12,5))
ax = sns.distplot(train_data['weight'],
                 bins=1400, 
                 kde_kws={"clip":(0.001, 1.4)},
                 hist_kws={"range":(0.001,1.4)},
                 color='darkcyan',
                 kde=False);

#次に、わかりやすくするため、値の大きさに合わせて色を付ける
#色を付けるときの一連の流れは同じ、values→Normalize→colors→rec,col→show
#values
values = np.array([rec.get_height() for rec in ax.patches])
#Normalize
norm = plt.Normalize(values.min(), values.max())
#colors
colors = plt.cm.jet(norm(values))
#rec,col
for rec,col in zip(ax.patches, colors):
    rec.set_color(col)
#show
plt.xlabel("Histogram of non-zero weights", size=14)
plt.show();
del values
gc.collect();

2つのピークがある  
①weight=0.17：高く狭い  
②weigh=0.34 ：低く広い
0以外のヒストグラム：peekは2つ(0.17:高く狭い,0.34:低く広い)
→2つの分布が重なっている（販売と購入？）


In [ ]:
#weightsの対数分布
#weightが0でないデータを格納
#query：指定した条件に合うものを取り出す
#reset_index:indexを連番に振り直す
#デフォルトでは、「元のindex」の列が残るが、drop=Trueとすると「元のindex」は消える
train_data_nonZero = train_data.query('weight > 0').reset_index(drop = True)

#plot
plt.figure(figsize = (10,4))
#ここで対数化する
ax = sns.distplot(np.log(train_data_nonZero['weight']),
                 bins=1000,
                 kde_kws={"clip":(-4,5)},
                 hist_kws={"range":(-4,5)},
                 color='darkcyan',
                 kde=False);

#色を付けるときの一連の流れは同じ、values→Normalize→colors→rec,col→show
values = np.array([rec.get_height() for rec in ax.patches])
norm = plt.Normalize(values.min(),values.max())
colors = plt.cm.jet(norm(values))
for rec, col in zip(ax.patches, colors):
    rec.set_color(col)
plt.xlabel("Histogram of the logarithm of the non-zero weights", size=14)
plt.show();
gc.collect();


In [ ]:
#ガウス分布

#再び曲線近似
from scipy.optimize import curve_fit

#the values
#x=0,1,2,...
x = list(range(len(values)))
#x = (x/110)-4
x = [(i/110)-4 for i in x]
y = values

#define a Gaussian function
#ガウス関数の定義
def Gaussian(x, mu,sigma, A):
    return A*np.exp(-0.5 * ((x-mu)/sigma)**2)

#二峰［双峰］性関数の定義
def bimodal(x, mu_1, sigma_1, A_1, mu_2, sigma_2, A_2):
    return Gaussian(x, mu_1, sigma_1, A_1) + Gaussian(x, mu_2, sigma_2, A_2)

#seed guess
initial_guess = (1, 1, 1, 1, 1, 1)

#the fit
#双峰性関数で曲線近似
parameters, covariance = curve_fit(bimodal, x, y, initial_guess)
sigma = np.sqrt(np.diag(covariance))

#the plot
plt.figure(figsize=(10,4))
#対数をとる
ax = sns.distplot(np.log(train_data_nonZero['weight']),
                 bins=1000,
                 kde_kws={"clip":(-4,5)},
                 hist_kws={"range":(-4,5)},
                 color='darkcyan',
                 kde=False)

#色を付けるいつものやつ
values = np.array([rec.get_height() for rec in ax.patches])
norm = plt.Normalize(values.min(), values.max())
colors = plt.cm.jet(norm(values))
for rec, col in zip(ax.patches, colors):
    rec.set_color(col)
plt.xlabel("Histogram of the logarithm of the non-zero weights", size=14)

#plot gaussian #1:bimodalのmu_1, sigma_1, A_1を用いる(・・)
plt.plot(x, Gaussian(x, parameters[0], parameters[1], parameters[2]), ':', color='black', lw=2, label='Gaussina #1', alpha=0.8)

#plot gaussian #2:bimodalのmu_2, sigma_2, A_2を用いる(---)
plt.plot(x, Gaussian(x, parameters[3], parameters[4], parameters[5]), '--', color='black', lw=2, label='Gaussina #2', alpha=0.8)

#plot the two gaussians toghther:上二つを合わせたもの(実線)
plt.plot(x, bimodal(x,*parameters), color='black', lw=2, alpha=0.7)

#凡例
plt.legend(loc="upper left");

plt.show();
del values
gc.collect();

対数分布、ガウス分布でみると、狭い左のpeekは違う分布  
μ平均  
小さいガウス：-1.32　大きいガウス：0.4
## 4,Cumulative return：累積daily return＝weight * calue of resp
resp_123→保守的conservative
Cumulative return のヒストグラム

In [ ]:
#各respについて、新しくweight*respのcolumnをつくる
train_data['weight_resp'] = train_data['weight']*train_data['resp']
train_data['weight_resp_1'] = train_data['weight']*train_data['resp_1']
train_data['weight_resp_2'] = train_data['weight']*train_data['resp_2']
train_data['weight_resp_3'] = train_data['weight']*train_data['resp_3']
train_data['weight_resp_4'] = train_data['weight']*train_data['resp_4']

#figureとAxesSubplotの準備
fig, ax = plt.subplots(figsize=(15,5))
#一次元化
#groupbyは、同じ値を持つデータを集め、それぞれの塊に対し、同じ操作を行う
#'date'で分け(同じ日付ごとのデータの塊が合計日数分できる)、
#それぞれの塊におけるweight_respの平均値（つまり一日の平均値)を計算し、
#それらに1を足して累積積を格納
resp = pd.Series(1+(train_data.groupby('date')['weight_resp'].mean())).cumprod()
#resp_1,2,3,4に対しても同様に行う
resp_1 = pd.Series(1+(train_data.groupby('date')['weight_resp_1'].mean())).cumprod()
resp_2 = pd.Series(1+(train_data.groupby('date')['weight_resp_2'].mean())).cumprod()
resp_3 = pd.Series(1+(train_data.groupby('date')['weight_resp_3'].mean())).cumprod()
resp_4 = pd.Series(1+(train_data.groupby('date')['weight_resp_4'].mean())).cumprod()

##このAxesSubplotのxラベルをつける(下の図でついて無くね？)
ax.set_xlabel("Day", fontsize=18)
ax.set_title("Cumulative daily return for resp and time horizons1,2,3, and 4(500days)", fontsize=18)
resp.plot(lw=3, label='resp x weight')
resp_1.plot(lw=3, label='resp_1 x weight')
resp_2.plot(lw=3, label='resp_2 x weight')
resp_3.plot(lw=3, label='resp_3 x weight')
resp_4.plot(lw=3, label='resp_4 x weight')

#day 85 marker
#垂直線axvline(alpha:透明度)
ax.axvline(x=85, linestyle='--', alpha=0.3, c='red', lw=1)
#一定区間に背景色を付ける
#sns.xkcd_rgbで、954色を自由に取り出せる
ax.axvspan(0, 85, color=sns.xkcd_rgb['grey'], alpha=0.1)
#凡例
plt.legend(loc="lower left")

最短のtime horizonsはresp_1,2,3で保守的な戦略はリターンが低いことを表している。

In [ ]:
#weightのヒストグラムをplotする
#さっきと同じように、weightが0でないデータを格納(query,reset：条件に合うものを取り出し連番に)
train_data_no_0 = train_data.query('weight > 0').reset_index(drop = True)
#新しいcolumnを作る、wAbsResp=weight*resp
train_data_no_0['wAbsResp'] = train_data_no_0['weight'] * (train_data_no_0['resp'])

#plot(histgram)
plt.figure(figsize = (12,5))
ax = sns.distplot(train_data_no_0['wAbsResp'],
                 bins=1500,
                 kde_kws={"clip":(-0.02,0.02)},
                 hist_kws={"range":(-0.02, 0.02)},
                 color='darkcyan',
                 kde=False);
#COLOR：value,norm,colors,rec/col
values = np.array([rec.get_height() for rec in ax.patches])
norm = plt.Normalize(values.min(), values.max())
colors = plt.cm.jet(norm(values))
#
for rec, col in zip(ax.patches, colors):
    rec.set_color(col)
plt.xlabel("Histogram of the weights * resp", size=14)
plt.show()

## 5,Time：各日の、ts_idの数をplot

垂直破線：85日目付近でモデルを修正している可能性があるため  

In [ ]:
#各日のtradeの回数の変数をつくる（日付でgroup分けし、ts_idの数を数える）
trades_per_day = train_data.groupby(['date'])['ts_id'].count()

#plot(折れ線グラフ)
fig, ax = plt.subplots(figsize=(15,5))
plt.plot(trades_per_day)
ax.set_xlabel("Day", fontsize=18)
ax.set_title("Total number of ts_id for each day", fontsize=18)

#day 85 marker
#x=85の点線
ax.axvline(x=85, linestyle='--', alpha=0.3, c='red', lw=1)
#その範囲をグレーにする
ax.axvspan(0,85, color=sns.xkcd_rgb['grey'], alpha=0.1)
#軸の範囲を設定
#xlim:軸の範囲の制限
#ax.set_:axのパラメータを後から変更できる
ax.set_xlim(xmin=0)
ax.set_xlim(xmax=500)
#notebook系では現在の図を勝手にplotしてくれるので、plt.show()は必要なく感じる
#複数のFigureを最後にplotしたり、途中でplotしたりするときに使える
plt.show()

In [ ]:
#trading day(一日の取引時間)は6.5hours(23400seconds)とする

#Average time between trades for each day
fig, ax = plt.subplots(figsize=(15,5))
#平均取引時間 = 時間/取引回数
plt.plot(23400/trades_per_day)
ax.set_xlabel("Day", fontsize=18)
ax.set_ylabel("Av. time between trades(s)", fontsize=18)
ax.set_title("Average time between trades for each day", fontsize=18)
ax.axvline(x=85, linestyle='--', alpha=0.3, c='red', lw=1)
ax.axvspan(0, 85, color=sns.xkcd_rgb['grey'], alpha=0.1)
ax.set_xlim(xmin=0)
ax.set_xlim(xmax=500)
ax.set_ylim(ymin=0)
ax.set_ylim(ymax=12)
plt.show()

In [ ]:
#histogram of the number of trades per day
plt.figure(figsize=(12,4))

#the minimum has been set to 1000 so as not to draw the partial days like day 2 and day 294
#the maximum number of trade per day is 18884
#I have used 125 bins for the 500 days
#→(1000,20000)
#histgram
ax = sns.distplot(trades_per_day,
                 #階級幅
                 bins=125,
                 kde_kws={"clip":(1000,20000)},
                 hist_kws={"range":(1000,20000)},
                 color='darkcyan',
                 # kde;密度近似関数の描画=True
                 kde=True);
values = np.array([rec.get_height() for rec in ax.patches])
norm = plt.Normalize(values.min(), values.max())
colors = plt.cm.jet(norm(values))
for rec, col in zip(ax.patches, colors):
    rec.set_color(col)
plt.xlabel("Number of trades per day", size=14)
plt.show()

In [ ]:
#上記より、maxで9000回取引があった日が複数あるのでみてみる
#volitile=volatile：不安定な；外れ値
volitile_days = pd.DataFrame(trades_per_day[trades_per_day > 9000])
volitile_days.T #転置

#不安定な日：9000回以上の取引があることもあり、ほとんどは85日目より前  
#feature_64もtimeに関連（※後述）

## 6,The features
### feature_0

In [ ]:
#feature_0
#唯一 +1 or -1であり、他と違いそうなのでまず見てみる

train_data['feature_0'].value_counts()

In [ ]:
#feature_0は、features.csv fileの中で唯一Trueを持たない
#features.csv：feature × Tagで、TFのみをとる
#→feature_0は、どのTagにも属さない

#折れ線plot
fig, ax = plt.subplots(figsize=(15,4))
#train_dataのfeature_0の一次元データの累積値を格納
feature_0 = pd.Series(train_data['feature_0']).cumsum()
ax.set_xlabel("Trade", fontsize=18)
ax.set_ylabel("feature_0(cumulative)", fontsize=18);
feature_0.plot(lw=3)

In [ ]:
#+1と-1の累積値で、最終的には,1207005-1183486=23519
#安定的に増加

#また、feature_0=1、feature_0=-1それぞれにおいて、culmative respとreturnは関連あり
## "An observation about feature_0" by therocket290
#さっきと同じように、feature_0が1(-1)のデータを格納(query,reset：条件に合うものを取り出し連番に)
feature_0_is_plus_one = train_data.query('feature_0 == 1').reset_index(drop = True)
feature_0_is_minus_one = train_data.query('feature_0 == -1').reset_index(drop = True)

#the plot
#2(1*2)つの図を用意(feature0 = 1or-1を用意)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,4))
#respをplot
ax1.plot((pd.Series(feature_0_is_plus_one['resp']).cumsum()), lw=3, label='resp')
ax2.plot((pd.Series(feature_0_is_minus_one['resp']).cumsum()), lw=3, label='resp')
#return(resp*weight)をplot
ax1.plot((pd.Series(feature_0_is_plus_one['resp']*feature_0_is_plus_one['weight']).cumsum()), lw=3, label='return')
ax2.plot((pd.Series(feature_0_is_minus_one['resp']*feature_0_is_minus_one['weight']).cumsum()), lw=3, label='return')
#title,legend
#ちなみに、legend=伝説、読まれるもの、昔話、集める(lect)、話す、説教、説明、凡例という語源
ax1.set_title("feature 0 = 1", fontsize=18)
ax2.set_title("feature 0 = -1", fontsize=18)
ax1.legend(loc="lower left")
ax2.legend(loc="lower left");

del feature_0_is_plus_one
del feature_0_is_minus_one
gc.collect();


+1 と-1　は全然違うreturnの動きをする。  
"Feature 0, beyond feature 0" written by NanoMathias a uniform manifold     approximation and projection (UMAP)   
→feature0は他のfeaturesを2つのdistributionsに分ける 
 
feature_0の値の違い  
→featureの2つの分布；売買(bid/ask, long/short, call/put)  

"What is "feature_0" ?"   

Lee and Ready 'Tick' model  
個々の取引を、日中の取引データと見積もりデータを用いて  
market buy orderかmarket sell orderかに分類  
buy始まりの取引を1とし、sell始まりの取引を-1とする

("Advances in Financial Machine Learning" by Marcos Lopez de Prado)   
buy始まりの取引を1とし、sell始まりの取引を-1とする

pt：時間t=1,...Tでのprice b0は任意  
  
Δpt > 0 ならbt = 1 (market buy order:buy-initiated trade)  
Δpt < 0 ならbt = -1(market sell order:sell-initiated trade)  
Δpt = 0 ならbt-1 = 0（価格pが同じときは一期前と同じ）  
（日中取引intraday trade ＆ 見積りquoteデータ）  
  
correlation matrix
→feature_0とTag12,(24)に正の相関、Tag13,(25,27)に強い負の相関  
→feature_37,38,39,40以外はすべてresp(特にresp4)と相関強い  
 


In [ ]:
#85日目でデータを分けるために、85日目の一番最初の取引データを取得
day85 = train_data[train_data.date == 85]
day85[day85.ts_id == day85.ts_id.min()]

### feature_1...129

In [ ]:
#feature_1,...129：特徴は4つに分けられそう  

#2*2の4つのplot
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20,10))

#①Linear:feature_1
ax1.plot((pd.Series(train_data['feature_1']).cumsum()), lw=3, color='red')
ax1.set_title("Linear", fontsize=22);
ax1.axvline(x=514052, linestyle='--', alpha=0.3, c='green', lw=2)
ax1.axvspan(0, 514052, color=sns.xkcd_rgb['grey'], alpha=0.1)
#端っこを0にする
ax1.set_xlim(xmin=0)
#ax1.set_xlabel("Trade", fontsize=18)
ax1.set_ylabel("feature_1", fontsize=18);

#②Noisy:feature_3
ax2.plot((pd.Series(train_data['feature_3']).cumsum()), lw=3, color='green')
ax2.set_title("Noisy", fontsize=22);
ax2.axvline(x=514052, linestyle='--', alpha=0.3, c='red', lw=2)
ax2.axvspan(0, 514052, color=sns.xkcd_rgb['grey'], alpha=0.1)
ax2.set_xlim(xmin=0)
#ax2.set_xlabel("Trade", fontsize=18)
ax2.set_ylabel("feature_3", fontsize=18);

#③Hybryd(Tag21):feature_55
ax3.plot((pd.Series(train_data['feature_55']).cumsum()), lw=3, color='darkorange')
ax3.set_title("Hybryd(Tag 21)", fontsize=22);
ax3.axvline(x=514052, linestyle='--', alpha=0.3, c='red', lw=2)
ax3.axvspan(0, 514052, color=sns.xkcd_rgb['grey'], alpha=0.1)
ax3.set_xlim(xmin=0)
ax3.set_xlabel("Trade", fontsize=18)
ax3.set_ylabel("feature_3", fontsize=18);

#④Negative：feature_73
ax4.plot((pd.Series(train_data['feature_73']).cumsum()), lw=3, color='blue')
ax4.set_title("Negative", fontsize=22);
#ax4.axvline(x=514052, linestyle='--', alpha=0.3, c='red', lw=2)
#ax4.axvspan(0, 514052, color=sns.xkcd_rgb['grey'], alpha=0.1)
#ax4.set_xlim(xmin=0)
ax4.set_xlabel("Trade", fontsize=18)
ax4.set_ylabel("feature_3", fontsize=18);
gc.collect();



### ①Linear：Tag14,18,22 
* 1
* 7,9,11,13,15
* 17,19,21,23,25
* 18,20,22,24,26
* 27,29,21,33,35
* 28,30,32,34,36
* 84,85,86,87,88
* 90,91,92,93,94
* 96,97,98,99,100
* 102(変化の勾配大),103,104,105,106

* 41,46,47,48,489,50,51,53,54,69,89,95(変化の勾配大),101,107(変化の勾配大),108.110,111,113,114,115,116,117,118,119(変化の勾配大),120,122,124も同様

### Features 41,42,43(Tag14)

In [ ]:
# Tag14はstratified（層化）している
# →discreate values（離散値）のみ採用（security；証券）
# day0,1,3のfeature_41,42,43をplot(day2を省く理由は後述)

#day0,1,3のcolumnをつくる　
#.loc:要素抽出：ラベル名可、絶対座標不可、複数要素指定可
#.at         ：ラベル名可、絶対座標不可、複数要素指定不可
#.iloc(iat)　 :ラベル名不可、絶対座標可
day_0 = train_data.loc[train_data['date'] == 0]
day_1 = train_data.loc[train_data['date'] == 1]
day_3 = train_data.loc[train_data['date'] == 3]

#3日間のデータを結合
three_days = pd.concat([day_0, day_1, day_3])
#散布図
#s:size(default=20)
three_days.plot.scatter(x='ts_id', y='feature_41', s=0.5, figsize=(15,3));
three_days.plot.scatter(x='ts_id', y='feature_42', s=0.5, figsize=(15,3));
three_days.plot.scatter(x='ts_id', y='feature_43', s=0.5, figsize=(15,3));

#del day_0
del day_1
del day_3
gc.collect();

In [ ]:
# feature_41,42,43はlagのplotも面白い
# ts_id(n)とts_id(n+1)を比べる

fig, ax = plt.subplots(1, 3, figsize=(17,4))
#pandas.plotting(plt).lag_plot：時系列でlagをplotできる
#(series=, lag=ずれ(defaultは1), ax;plotするaxis object)
#sは多分size
lag_plot(day_0['feature_41'], lag=1, s=0.5, ax=ax[0])
lag_plot(day_0['feature_42'], lag=1, s=0.5, ax=ax[1])
lag_plot(day_0['feature_43'], lag=1, s=0.5, ax=ax[2])

#feature41
ax[0].title.set_text('feature_41')
ax[0].set_xlabel("ts_id(n)")
ax[0].set_ylabel("ts_id(n+1)")

#feature42
ax[1].title.set_text('feature_42')
ax[1].set_xlabel("ts_id(n)")
ax[1].set_ylabel("ts_id(n+1)")

#feature43
ax[2].title.set_text('feature_43')
ax[2].set_xlabel("ts_id(n)")
ax[2].set_ylabel("ts_id(n+1)")

#(0.0)に赤い点
ax[0].plot(0, 0, 'r.', markersize=15.0)
ax[1].plot(0, 0, 'r.', markersize=15.0)
ax[2].plot(0, 0, 'r.', markersize=15.0)
gc.collect();


### Features_44(tag15),45(tag17)

In [ ]:
# これらのfeaturesは上に見られるtag14に似ているが、より0を中心としている

#ts_idとfeature_44,45を同様にplot
three_days.plot.scatter(x='ts_id', y='feature_44', s=0.5, figsize=(15,3));
three_days.plot.scatter(x='ts_id', y='feature_45', s=0.5, figsize=(15,3));
gc.collect();

In [ ]:
#同様にlagをplot
fig, ax = plt.subplots(1, 2, figsize=(15,4))
lag_plot(day_0['feature_44'], lag=1, s=0.5, ax=ax[0])
lag_plot(day_0['feature_45'], lag=1, s=0.5, ax=ax[1])


#feature41
ax[0].title.set_text('feature_44')
ax[0].set_xlabel("ts_id(n)")
ax[0].set_ylabel("ts_id(n+1)")

#feature42
ax[1].title.set_text('feature_45')
ax[1].set_xlabel("ts_id(n)")
ax[1].set_ylabel("ts_id(n+1)")

#(0.0)に赤い点
ax[0].plot(0, 0, 'r.', markersize=15.0)
ax[1].plot(0, 0, 'r.', markersize=15.0)
gc.collect();


### Features 60 to 68(Tag22)

In [ ]:
# 60,61,62,63,64,65,66,67,68

#これらのfeaturesの累積グラフ

fig, ax = plt.subplots(figsize=(15,5))
feature_60 = pd.Series(train_data['feature_60']).cumsum()
feature_61 = pd.Series(train_data['feature_61']).cumsum()
feature_62 = pd.Series(train_data['feature_62']).cumsum()
feature_63 = pd.Series(train_data['feature_63']).cumsum()
feature_64 = pd.Series(train_data['feature_64']).cumsum()
feature_65 = pd.Series(train_data['feature_65']).cumsum()
feature_66 = pd.Series(train_data['feature_66']).cumsum()
feature_67 = pd.Series(train_data['feature_67']).cumsum()
feature_68 = pd.Series(train_data['feature_68']).cumsum()
#feature_69 = pd.Series(train_data['feature_69']).cumsum()

ax.set_xlabel("Trade", fontsize=18)
ax.set_title("Cumulative plot for feature_60,...,68(Tag22).", fontsize=18)

feature_60.plot(lw=3)
feature_61.plot(lw=3)
feature_62.plot(lw=3)
feature_63.plot(lw=3)
feature_64.plot(lw=3)
feature_65.plot(lw=3)
feature_66.plot(lw=3)
feature_67.plot(lw=3)
feature_68.plot(lw=3)
#feature_69.plot(lw=3)

plt.legend(loc="upper left");
del feature_60, feature_61, feature_62, feature_63, feature_64, feature_65, feature_66, feature_67, feature_68
gc.collect();

In [ ]:
#上のグラフにおいて、feature_60と61,62と63,65と66,67と68は似ているので、
#それらのdistribution(分布)をplot

#defaultでデフォルトのカラーパレットは、
#6つのテーマ(deep, muted, pastel, bright, dark, colorblind)がある
sns.set_palette("bright")

fig, axes = plt.subplots(2,2,figsize=(8,8))

#feature60と61
#ヒストグラム(階級幅が今までと比べ一桁小さい)
sns.distplot(train_data[['feature_60']], hist=True, bins=200, ax=axes[0,0])
sns.distplot(train_data[['feature_61']], hist=True, bins=200, ax=axes[0,0])
axes[0,0].set_title("feature 60 and 61", fontsize=18)
axes[0,0].legend(labels=['60', '61'])

#feature62と63
sns.distplot(train_data[['feature_62']], hist=True, bins=200, ax=axes[0,1])
sns.distplot(train_data[['feature_63']], hist=True, bins=200, ax=axes[0,1])
axes[0,1].set_title("feature 62 and 63", fontsize=18)
axes[0,1].legend(labels=['62', '63'])

#feature65と66
sns.distplot(train_data[['feature_65']], hist=True, bins=200, ax=axes[1,0])
sns.distplot(train_data[['feature_66']], hist=True, bins=200, ax=axes[1,0])
axes[1,0].set_title("feature 65 and 66", fontsize=18)
axes[1,0].legend(labels=['65', '66'])

#feature67と68
sns.distplot(train_data[['feature_67']], hist=True, bins=200, ax=axes[1,1])
sns.distplot(train_data[['feature_68']], hist=True, bins=200, ax=axes[1,1])
axes[1,1].set_title("feature 67 and 68", fontsize=18)
axes[1,1].legend(labels=['67', '68'])

plt.show();
gc.collect();

In [ ]:
#それらの間にあるfeature_64を見てみる
plt.figure(figsize = (12,5))
ax = sns.distplot(train_data['feature_64'],
                  bins=1200,
                  kde_kws={"clip":(-6,6)},
                  hist_kws={"range":(-6,6)},
                  color='darkcyan',
                  kde=False);
values = np.array([rec.get_height() for rec in ax.patches])
norm = plt.Normalize(values.min(), values.max())
colors = plt.cm.jet(norm(values))
for rec, col in zip(ax.patches, colors):
    rec.set_color(col)
plt.xlabel("Hisogram of feature_64", size=14)
plt.show();
del values
gc.collect();

上の図において、0.7から1.38の間に大きなgapがある  
ちなみにlog2=0.693、log4=1.386だが、これに意味があるかは不明

In [ ]:
#Tag22のfeatures(60-68)はdaily patternも興味深い
#例えば、feature64の、3日間のscatter(分散)とcumulative(累積)をplot

day_0 = train_data.loc[train_data['date'] == 0]
day_1 = train_data.loc[train_data['date'] == 1]
day_3 = train_data.loc[train_data['date'] == 3]
three_days = pd.concat([day_0, day_1, day_3])

#plot
##sharex:複数グラフ間の軸を共有(x軸)
fig, ax = plt.subplots(2, 1, figsize=(15,6), sharex=True)

ax[0].scatter(three_days.ts_id, three_days.feature_64, s=0.5, color='b')
ax[0].set_xlabel('') #下の図と共有
ax[0].set_ylabel('value')
ax[0].set_title('feature_64(days 0, 1 and 3)')

ax[1].scatter(three_days.ts_id, pd.Series(three_days['feature_64']).cumsum(), s=0.5, color='r')
ax[1].set_xlabel('ts_id')
ax[1].set_ylabel('cumulative sum')
ax[1].set_title('') #上の図と共有

plt.show();


In [ ]:
#上図においてfeature_64のthe global minimum valueは-6.4
#feature_64のthe global maximum valueは8
print(train_data['feature_64'].max())
print(train_data['feature_64'].min())

In [ ]:
#NewYorkのStock Exchangeの一日の取引時間は9:30-16:00であることを考えてみる

#feature_64の単位が、およそ30minutesで、feature_64=0が12時に一致していたとすると、
#arcsin functionをplotでき、y軸を、一日の時間とすることができる

#np.arange:等差数列(start,stop,step) step:間隔
x = np.arange(-1, 1, 0.01)
#アークサイン
y = 2 * np.arcsin(x) + 1

#plot
fig, ax = plt.subplots(1, 1, figsize=(7,4))
ax.plot(x, y, lw=3)

#目盛ラベル(x軸;メモリ的な意味でのtime(つまりnp.arangeで作った-1～1) は無しにしておく)
ax.set(xticklabels=[])
ax.set(yticklabels=['9:00', '10:00', '11:00', '12:00', '13:00', '14:00', '15:00', '16:00'])

#$\it{}$ :斜体
#$\mathsf{}$：立体
#"$\<書体>{}$":上付き、下付き文字

ax.set_title("2$\it{arcsin}$(t) +1", fontsize=18)
ax.set_xlabel("'tick' time", fontsize=18)
ax.set_ylabel("Clock time", fontsize=18)
plt.show();

In [ ]:
#上図より、trade(その日におけるts_idの数)は、一日の中間よりも始まりから終わりの方がより頻繁
#仮想のtick frecuencyをplotする

#図をplot(t≠1,-1なので範囲はそれをふまえる(-0.98がなぜダメかは不明))
#dash:微分した、という意味
x_dash = np.arange(-0.98, 0.99, 0.01)

#一個前のセルのやつの微分
#d/dt(2arcsin(t)+1) = 2/√(1-t^2)
y_dash = 2 / np.sqrt(1-(x_dash**2))

fig, ax = plt.subplots(1, 1, figsize=(7,4))
ax.plot(x_dash, y_dash, lw=3)

#yのlabelをなくす
ax.set(yticklabels=[])

#x軸を変更：時間区分を8つにするため、-1～1の2区間を、2/7=2.85より2.8ずつに分解
ax.xaxis.set_ticks(np.arange(-1, 1, 0.28))
#xのlabelを時間に変更
ax.set(xticklabels=["9:00", '10:00', '11:00', '12:00', '13:00', '14:00', '15:00', '16:00'])
ax.set_title("d/dt (2$\it{arcsin}$(t) +1)", fontsize=18)
ax.set_xlabel("Clock time", fontsize=18)
ax.set_ylabel("'tick' frequency", fontsize=18)
plt.show();




In [ ]:
#上のことを踏まえると、複数のfeatureにおいて
#一日の始まりに見られる欠損値のperiodは、
#一日の中間に見られる欠損値のperiodと同じなのか？？

#またおそらく、一日の始まりと終わりにおける高いthick frequencyは、
#夜中に重要なpositionがないようにするための
#open直後の大量の購入と、close直前の大量の売却によるものではないか？？

##marketneutral TOkyo Stock Exchange
#→日本株に関するものではないか
#日本のtrading hours は9:00-11:30,breakを挟んで12:30-15:00まで
##これによりTag22のfeaturesの中央の不連続性（gap）が説明できそう

#feature_65をplot
three_days.plot.scatter(x='ts_id', y='feature_65', s=0.5, figsize=(15,4));

## "Important and Hidden Temporal Data" written by Lachlan Suter.

### ②Noisy：Tag19  
* 3,4,5,6
* 8,10,12,14,16
* 37,38,39,40
* 72,73,74,75,76
* 78,79,80,81,82
* 83

In [ ]:
#これらのfeaturesの一部のcumulative plots

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16,8))

#以下の「+Tag9」というのはTag9がFalseの時とTrueの時を比べて、ということ
#例えば、feature3,5,37,39はFalseでfeature4,6,38,40はTrue

#feature3,4
ax1.set_title("features 3 and 4 (+Tag 9)", fontsize=18);
ax1.plot((pd.Series(train_data['feature_3']).cumsum()), lw=2, color='blue')
ax1.plot((pd.Series(train_data['feature_4']).cumsum()), lw=2, color='red')

#feature5,6
ax2.set_title("features 5 and 6 (+Tag 9)", fontsize=18);
ax2.plot((pd.Series(train_data['feature_5']).cumsum()), lw=2, color='blue')
ax2.plot((pd.Series(train_data['feature_6']).cumsum()), lw=2, color='red')

#feature37,38
ax3.set_title("features 37 and 38 (+Tag 9)", fontsize=18);
ax3.plot((pd.Series(train_data['feature_37']).cumsum()), lw=2, color='blue')
ax3.plot((pd.Series(train_data['feature_38']).cumsum()), lw=2, color='red')

#feature39,40
ax4.set_title("features 39 and 40 (+Tag 9)", fontsize=18);
ax4.plot((pd.Series(train_data['feature_39']).cumsum()), lw=2, color='blue')
ax4.plot((pd.Series(train_data['feature_40']).cumsum()), lw=2, color='red')
ax4.axvline(x=514052, linestyle='--', alpha=0.3, c='green', lw=2)
ax4.axvspan(0, 514052, color=sns.xkcd_rgb['grey'], alpha=0.1)
ax4.set_xlabel("Trade", fontsize=18)

gc.collect();

これら(features？)はoffer pricesとTag９のbid pricesを表すことができるか？  　
* offer price:提示価格＝売値
* bid price　 :入札価格＝買値

85日の後のfeature_40の値は実際にfeature_39の値よりも大きくなっている　  
(offer > bid は常 ：差＝スプレッドspread(会社によって違い、変動することもある))

### feature_51(Tag19)
Weight and feature_51 de-anonymized" by marketneutral   
→weightはtradeで予想されるtransaction cost(取引コスト)  
→weightがrespの分散に反比例  
→weightは簡単な取引コストの評価基準

→feature51は株式の一日の平均の出来高（の対数）と言われている


In [ ]:
#non-zero weightsに関して、feature_51を再構築してplot
# ※w.r.t.:with reference to(～に関して)

#分散
fig, ax = plt.subplots(figsize=(15,4))
#color:blue
ax.scatter(train_data_nonZero.weight, train_data_nonZero.feature_51, s=0.1, color='b')
ax.set_xlabel('weight')
ax.set_ylabel('feature_51')
plt.show()

### feature_52(tag19)

In [ ]:
#折れ線グラフ
fig, ax = plt.subplots(figsize=(15,3))
feature_0 = pd.Series(train_data['feature_52']).cumsum()
ax.set_xlabel("ts_id, fontsize=18")
ax.set_ylabel("feature_52(cumulative)",fontsize=12);
feature_0.plot(lw=3)

#lag plot
fig, ax = plt.subplots(1,1,figsize=(4,4))
lag_plot(day_0['feature_52'], s=0.5, ax=ax)
ax.title.set_text('feature_52')
ax.set_xlabel("ts_id(n)")
ax.set_ylabel("ts_id(n+1)")
#r.:ピリオド必須
##よくわからないけどredでは反応しない
ax.plot(0,0,'r.',markersize=15.0);

#respとも関係している
fig, ax = plt.subplots(figsize=(15,4))
ax.scatter(train_data_nonZero.feature_52, train_data_nonZero.resp, s=0.1, color='b')
ax.set_xlabel('feature_52')
ax.set_ylabel('resp')
plt.show();

### ③Hybryd：Tag21  
55,56,57,58,59  

In [ ]:
#これらはnoisyから始まる
##0.2M、0.5M、0.8M(month?)のtrade marks(取引のマーク)付近に顕著にほぼ不連続なステップがある
#そこから先はlinear(線形)
#これらの５つのfeaturesはTag21のsetを形成する

fig, ax = plt.subplots(figsize=(15,5))

feature_55 = pd.Series(train_data['feature_55']).cumsum()
feature_56 = pd.Series(train_data['feature_56']).cumsum()
feature_57 = pd.Series(train_data['feature_57']).cumsum()
feature_58 = pd.Series(train_data['feature_58']).cumsum()
feature_59 = pd.Series(train_data['feature_59']).cumsum()

ax.set_xlabel("Trade", fontsize=18)
ax.set_title("Cumulative plot for the 'Tag21' feature(55-59)", fontsize=18)

ax.axvline(x=514052, linestyle='--', alpha=0.3, c='black', lw=1)
ax.axvspan(0, 514052, color=sns.xkcd_rgb['grey'], alpha=0.1)

feature_55.plot(lw=3)
feature_56.plot(lw=3)
feature_57.plot(lw=3)
feature_58.plot(lw=3)
feature_59.plot(lw=3)
plt.legend(loc="upper left");
gc.collect();

これらが5つのrespの値に関係しているとすると？    
* feature_55はresp_1に関連
* feature_56はresp_4に関連
* feature_57はresp_2に関連
* feature_58はresp_3に関連
* feature_59はrespに関連

その場合は、  

* Tag0はresp_4のfeaturesを表す
* Tag1はrespのfeaturesを表す
* Tag2はresp_3のfeaturesを表す
* Tag3はresp_2のfeaturesを表す
* Tag4はresp_1のfeaturesを表す

つまり、  

* resp_1：7,8,17,18,27,28,55,72,78,84,90,96,102,108,114,120,121  
(すべてのmissing dataの79.6%がこのsetの中にある)
* resp_2：11,12,21,22,31,32,57,74,80,86,92,98,104,110,116,124,125  
(すべてのmissing dataの15.2%がこのsetの中にある)
* resp_3：13,14,23,24,33,34,58,75,81,87,93,99,105,111,117,,126,127
* resp_4：9,10,19,20,29,30,56,73,79,85,91,97,103,1099,115,122,123
* resp：15,16,25,26,35,36,59,76,82,88,94,100,106,112,118,128,129

In [ ]:
# 各respの17featuresのそれぞれをplot
# ※これはイメージで、正確には右クリックで拡大せよ

#Note:別のKaggleのnotebookからimport
#なぜなら、この図を作るのにほぼすべてのnotebookのmemoryを使うため。
#Image(filename="/kaggle/input/jane-17-plots/17_plots.png", width= "95%")
#どうやって開くのかよくわからない

縦の破線：day85  
  
resp_1,2,3,4：モデルトレーニングを正規化するための代替的な客観的metricsを必要とするときのためのもの  
→それを意図しないならば、おそらく4×17=68のfeatures全てを落として、
featuresに関係するrespを保持することになる

### ④Negative：Tag23
feature_73,75,76,77(noisy),79,81(noisy),82  

  
### t-SNE plots；t-distributed stohchastic neighbor embedding plots  

t分布の確率的隣接埋めこみプロット  

"Jane Street: t-SNE using RAPIDS cuML"  
計算に時間がかかるため、別のnotebookに作った

## 7,The features.csv file：匿名化されたfeaturesに関連するメタデータ
"metadata pretaining to the anonymized features"を含むfile  
1:True 0:False  
featureに関連づいたtag29個をもつ


feature→tag1～4個(feature_0:tag0個)

In [ ]:
feature_tags = pd.read_csv("../input/jane-street-market-prediction/features.csv", index_col=0)
#binary（二値）に変換;元々True or Falseなので0or1にする
feature_tags = feature_tags*1
#transoirted（転置された）dataframeをplot
#元のデータは行：feature、列：tag→行と列を入れ替え

#style:DataFrameに様々なカスタマイズ
#style.background_gradient:各セルの値に応じてカラーマップを適用する
feature_tags.T.style.background_gradient(cmap='Oranges')

In [ ]:
#上のデータを'8-bit' modeでre-plot
#全く判読不能だが、もしかしたら全体の視覚の助けとしての役割を果たすかもしれない
plt.figure(figsize=(32,14))
#ヒートマップ
#cbar:カラーバーを設定するか
sns.heatmap(feature_tags.T,
           cbar=False,
           xticklabels=False,
           yticklabels=False,
           cmap="Oranges");

In [ ]:
#それぞれのfeatureのtagの数を合計する
#DataFrameは、行がfeature、列がTag→転置しているので逆
#axis=0:行を順に見て行く
#列ごとに行の和を出す(featureごとにTagの和を出す)
#pd.DataFrame(feature_tags.sum(axis=1), columns=['Number of tags'])と同じ

tag_sum = pd.DataFrame(feature_tags.T.sum(axis=0), columns=['Number of tags'])
tag_sum.T

ほぼすべてのfeaturesは少なくとも１つのtagをもち、4つのtagを持つものもある  
唯一feature_0のみ全くtagをもたない  
それぞれ異なる、5(ish)regionsに分けられる   
ish:およそ  

  
| Region | features | Tags       | missing value? | observations        |
| ------ | -------- | ---------- | -------------- | ------------------- |
| 0      | 0        | none       | none           | -1or+1              |
| 1      | 1...6    | Tag6       |                |                     |
| 2      | 7...36   | Tag6       |                |                     |
| 3      | 37...72  | various    |                |                     |
| 4      | 72...119 | Tag23      |                |                     |
| 5      | 120...129| Tag28      |                |                     |
| ------ | -------- | ---------- | -------------- | ------------------- |
| 2      | 7...36   | Tag6       |                |                     |
| 2a     | 7...16   | +11        | 7,8 & 11,12    |                     |
| 2b     | 17...26  | +12        | 17,18 & 21,22  |                     |
| 2c     | 27...36  | +13        | 27,28 & 31,32  |                     |
| 3      | 37...72  | various    |                |                     |
| 3a     | 55...59  | Tag21      |                | All hybrid          |
| 3b     | 60...68  | Tag22      |                | Cloc+time features? |
| 4      | 72...119 | Tag23      |                |                     |
| 4a     | 72...77  | +15&27     | 72 & 74        |                     |
| 4b     | 78...83  | +17&27     | 78 & 80        |                     |
| 4c     | 84...89  | +15&25     | 84 & 86        |                     |
| 4d     | 90...95  | +17&25     | 90 & 92        |                     |
| 4e     | 96...101 | +15&24     | 96 & 98        |                     |
| 4f     | 102...107| +17&24     | 102 & 104      |                     |
| 4g     | 108...113| +15&26     | 108 & 110      |                     |
| 4h     | 114...119| +17&26     | 114 & 116      |                     |
| 5      | 120...129| Tag28      |                |                     |
| 5a     | 120      | +4         | missingdata    |                     |
| 5b     | 121      | +4&1       | missingdata    |                     |
| 5c     | 122      | +0         |                |                     |
| 5d     | 123      | +0&16      |                |                     |
| 5e     | 124      | +3         |                |                     |
| 5f     | 125      | +3&16      |                |                     |
| 5g     | 126      | +2         |                |                     |
| 5h     | 127      | +2&16      |                |                     |
| 5i     | 128      | +1         |                |                     |
| 5j     | 129      | +1&16      |                |                     |

上図には、どこよりも、'継続される...'という古典的な価値がある  

従来の表の形式で何が起こっているのかを明らかにすることはとても難しい  
🌐EDA: Tag Network Analysis (networkx + gephi)🌐"   
→tagsとfeaturesの間の関係を示すグラフ分析が実行されている  
  
Greg Calvez   
→tag_6：prices、tag_23：volume、tag_20：spreadに関連  
→tag_12：minima、tag_13：maxima、tag_22：timeに関連  

## 8,Action
このコンペの目的：action    
1:取引する、 0:取引しない、のbinary(2進)column  

In [ ]:
#1:取引する(resp=+), 0:取引しない(resp=-)
# 最後の*1によって、TorF→1or0に変換
train_data['action'] = ((train_data['resp'])>0)*1

#actionとinaction全体を比較
train_data['action'].value_counts()

In [ ]:
#上の式より、全体においてわずかにactiveが多い(0.4%)
#日ごとに見てみる

#actionのratio = 日ごとのaction(0or1)のsum / 日ごとのactionのcount（日数合計）
daily_action_sum   = train_data['action'].groupby(train_data['date']).sum()
daily_action_count = train_data['action'].groupby(train_data['date']).count()
daily_ratio        = daily_action_sum / daily_action_count


#plotする
fig, ax = plt.subplots(figsize=(15,5))
plt.plot(daily_ratio)
ax.set_xlabel("Day", fontsize=18)
ax.set_ylabel("ratio", fontsize=18)
ax.set_title("Daily ration of action to inaction", fontsize=18)

#y=0.5に直線の理由：action(1)とinaction(0)が1:1ならy=0.5のはず
#だから、実際にはわずかに上寄り
plt.axhline(0.5, linestyle='--', alpha=0.85, c='r');
ax.set_xlim(xmin=0)
ax.set_xlim(xmax=500)
plt.show()

In [ ]:
#日ごとのactionはかなり一貫性があり、
#明らかな週・月・季節等の変化はない

#平均
daily_ratio_mean = daily_ratio.mean()
#print('The mean daily ratio is %.3f' % daily_ratio_mean)
print('The mean daily ratio is {:3f}'.format(daily_ratio_mean))

#最大
daily_ratio_max = daily_ratio.max()
#print('The maximum daily ratio is %.3f' % daily_ratio_max)
print('The maximum daily ratio is {:3f}'.format(daily_ratio_max))


最大となるのはday294(後述)  
上記は明らかに複雑なデータセットにおけるとても単純化したtargetである
"Target Engineering; CV; ⚡ Multi-Target" written by marketneutral


## 9,The first day ("day 0")：missing dataにはpattternがある

In [ ]:
#day_0というdataframeをつくる
day_0 = train_data.loc[train_data['date'] == 0]

#そしてplot
fig, ax = plt.subplots(figsize=(15,5))
balance = pd.Series(day_0['resp']).cumsum()
resp_1 = pd.Series(day_0['resp_1']).cumsum()
resp_2 = pd.Series(day_0['resp_2']).cumsum()
resp_3 = pd.Series(day_0['resp_3']).cumsum()
resp_4 = pd.Series(day_0['resp_4']).cumsum()

ax.set_xlabel("Trade", fontsize=18)
ax.set_title("Cumulative values for resp and time horizons 1,2,3&4 for day0", fontsize=18)
balance.plot(lw=3)
resp_1.plot(lw=3)
resp_2.plot(lw=3)
resp_3.plot(lw=3)
resp_4.plot(lw=3)
plt.legend(loc="upper left")

In [ ]:
#Descriptive statistics of the 'train.csv' file for day 0(記述統計)

day_0.describe().style.background_gradient(cmap=colorMap)


## 10,Are there any missing values?


In [ ]:
#day_0からスタート

#misc
#欠損値を効果的に可視化する
#import missingno as msno
#msno.matrixでおおまかな欠損値の出現パターンがわかる
#sns.heatmapの情報に加えて、右側にバーが表示される(非欠損値の個数の線グラフ)

#day0における、行はts-id（全取引）、列はfeaturesなどのcolumn?(138じゃないの？)
#全ての行が68-144個の非欠損値をもつ
#color=(r,g,b)
msno.matrix(day_0, color=(0.35, 0.35, 0.75));

In [ ]:
#グラフ的に見ると、いくつかのcolumnsにmissing dataのchunks（塊）がある
#パターンがありそうなので、
#feature_7(resp1の最初のfeature)とfeature_11(resp2の最初のfeature)を見てみる
#.loc:要素抽出：ラベル名可、絶対座標不可、複数要素指定可
#.at         ：ラベル名可、絶対座標不可、複数要素指定不可
#.iloc(iat)　 :ラベル名不可、絶対座標可
#ちなみにat,iatの方が処理速度ははやい

#絶対座標の複数の要素の値を取得
#行の指定を「:(全体のスライス)」にすると列を参照できる(インデックス参照ではできない)
#列番号を指定
feats_7_11 = day_0.iloc[:, [14,18]]
msno.matrix(feats_7_11, color=(0.35, 0.35, 0.75), width_ratios=(1,3));

In [ ]:
#ランダムには見えない
#→2big chunks(各columnのはじめと途中)
#→trading day spans：9:30-16:00
#→missing data
#f_7:9:30-10:03,13:17-13:33(16m)
#f_11:9:30-9:35, 13:17-12:22(5.5m)

#train.csv file全体のそれぞれのcolumnのmissing dataの数の合計を見てみる
#数を集計
#.isna():欠損値ならTrue　→それのSUMより、欠損値の合計数になる
#.sort_values(ascending=False)：要素でソートする
#デフォルトは昇順、ascending=Falseで降順（欠損値数が多い順）
missing_data = pd.DataFrame(train_data.isna().sum().sort_values(ascending=False),columns=['Total missing'])
missing_data.T

In [ ]:
#視覚化
#欠損値の合計データを格納
gone = train_data.isnull().sum()

#棒グラフ、
#.values:辞書(key:values)のvalues(つまり欠損値の数)を取得
#値に応じてcolorをつける
px.bar(gone, color=gone.values, title="Total number of missing values for each columns").show()

missing data  
* Tag4(resp1)に79.6%
* Tag3(resp2)に15.2%、合わせて95%

resp1
* f_7,8：393135
* f_17,18,27,28：395535
* f_72,78,84,90,96,102,108,114：351426

resp2
* f_21,22,31,32：81444(f_11,12と近い)

よりmisssing valuesが少ないfeaturesがもっと存在する  
→しかし大事なのはmissing valuesの量よりも、   
似たmeasures/matrics（特徴）を示すfeaturesが教えてくれることである

## 11,Is there any missing data: Days 2 and 294
feature_64の分散をplotすると、どの日も同じ抜本的なパターンがある  
しかしday2は23のts_id、しかも一日の最後の方にある

day0以外にもmissing dayはあるが、patternはない  
取引数に応じて欠損値数をplot→day2と294を除いて平均的

In [ ]:
#day0が特別なのか、毎日missing dataはあるのか？
#列番号7:137のデータをiloc
#to_frame:すぐにデータフレーム化
missing_features = train_data.iloc[:, 7:137].isnull().sum(axis=1).groupby(train_data['date']).sum().to_frame()

#plot
fig, ax = plt.subplots(figsize=(15,5))
plt.plot(missing_features)
ax.set_xlabel("Day", fontsize=18)
ax.set_title("Total number of missing values in all features for each day", fontsize=18)
ax.axvline(x=85, linestyle='--', alpha=0.3,c='red', lw=2)
ax.axvspan(0, 85, color=sns.xkcd_rgb['grey'], alpha=0.1)
ax.set_xlim(xmin=0)
ax.set_xlim(xmax=500)
plt.show()


In [ ]:
#missing data はほぼ毎日見られ、discernible（識別可能な）pattern(weekly,monthly...)はない
#day2,day294は例外

# "Jane Street EDA Market Regime"
#日ごとのtrade数をplot(大体3000～!)
#驚くほど上のplotと似ている
#毎日のtrade数に関してmissing valuesの数をplot

#日ごとのtrade数のcolumn
count_weights = train_data[['date', 'weight']].groupby('date').agg(['count'])
#日ごとのmissing data数のcolumnと結合
result = pd.merge(count_weights, missing_features, on = "date", how="inner")
#結合した後、column名を付け直す
result.columns = ['weights', 'missing']
#取引数に対するmissingの割合をratioとする
result['ratio'] = result['missing'] / result['weights']
#平均
missing_per_trade = result['ratio'].mean()

#plot
fig, ax = plt.subplots(figsize=(15,5))
plt.plot(result['ratio'])
plt.axhline(missing_per_trade, linestyle='--', alpha=0.85, c='r');
ax.set_xlabel("Day", fontsize=18)
ax.set_title("Average number of missing feature values per trade,for each day", fontsize=18)
plt.show()

一日に、取引ごとに平均で3つのmissing featureがある(day2,294を除く)  
day2,294には全くmissing valueがない  
dya14がmissing valuesが最も多い  

→目に見えないtest dataのmissing dataをどうすればよいか問題  
何をするにしても、コンペでは時間が重要なので早くやる必要あり  
"Optimise Speed of Filling-NaN Function"(さらに詳しく)

## 11,Is there any missing data: Days 2 and 294

In [ ]:
#2日目にマークをするために、2日目の一番最初の取引データを取得
day2 = train_data[train_data.date == 2]
min2 = day2.ts_id.min()
max2 = day2.ts_id.max()
mean2 = day2.ts_id.mean()
#(min+max)_2と同じ

print(min2, max2, mean2)
#なぜ次で15150を持ってきたかは不明

In [ ]:
#feature_64の散布図を作ると、それぞれの日に同じsweeping(抜本的)　patternがある 
#しかしdayには231のts_idしかなく、それは一日の本当に最後から始まっている

#day1,2,3(1,3:blue 2:red)のplot
day_1 = train_data.loc[train_data['date'] ==1]
day_2 = train_data.loc[train_data['date'] ==2]
day_3 = train_data.loc[train_data['date'] ==3]
three_days = pd.concat([day_1, day_2, day_3])

td = three_days.plot.scatter(x='ts_id', y='feature_64', s=0.5, figsize=(15,4), color='blue')
day_2.plot.scatter(x='ts_id', y='feature_64', s=0.5, figsize=(15,4), color='red', ax=td);

#さらにわかりやすく
fig,ax = plt.subplots(figsize=(15,3))
ax.scatter(three_days.ts_id, three_days.feature_64, s=0.5, color='b')
ax.scatter(day_2.ts_id, day_2.feature_64, s=0.5, color='r')
#(15150,5.2のところに、size=1800,facecolor(円の中の色)=noneの黒の点線の円をかく)
ax.scatter(15150, 5.2, s=1800, facecolors='none', edgecolors='black', linestyle='--', lw=2)
ax.set_xlabel('feature_64')
ax.set_ylabel('ts_id')
ax.set_title('feature_64 for days 1, 2 and 3')

days294も同様になり、ts_idは29個しかない  
これも、なぜ他の日の朝食や昼食の時間に見られるmissing valuesが存在しないのかを説明している  
→外れ値としてその2日をdropすることが価値のある処理となる可能性がある


## 12,DABL plots (targets: action and resp)
：automatic plots for classification&regression
actionの**0,1はwell balanced

In [ ]:
#dabl:データ分析のベースライン的なライブラリ
#dablを用いてday0を実行
#まずはactionをplot
#col=column
dabl.plot(day_0, target_col="action")

In [ ]:
#actionの0と1のクラスはほどほどに(reasonably)良いバランスである
#次にrespを見てみる
dabl.plot(day_0, target_col="resp")

In [ ]:
#day0において、time(ts_is)に関してresp valuesをplot
#trendline;近似曲線 ols:Ordinary Least Squares(最小二乗回帰)
#marginal:周辺的、欄外のplot(histgramなどもできる)
#viloin:データの分布密度を視覚化
fig_1 = px.scatter(day_0, x=day_0['ts_id'], y=day_0['resp'],
                  trendline="ols", marginal_y="violin",
                  title=("Scatter plot of resp with respect tp ts_id for day 0"))
fig_1.show()


## 13,Permutation(順序) Importance using the Random Forest


In [ ]:
#Very quick Permutation Importance using the Random Forest
#Random Forestを用いたとてもはやい順序の重要性

## permutation importance 
#どのfeaturesが重要か、はとても大事。それををみる基本的な方法
#この方法は早く、広く使われ理解されやすく、
#feature　importanceの尺度に必要なproperty(性質?)と一致しているため
#modelがfitされた後に計算される
#respをtargetとしてregression(回帰)を実行

#.contains:特定の文字列を含む行(:→列)を抽出
X_train = day_0.loc[:, day_0.columns.str.contains('feature')]
#fillna:欠損値を平均で穴埋め
X_train = X_train.fillna(X_train.mean())

#targetはaction
y_train = day_0['resp']

from sklearn.ensemble import RandomForestRegressor
#RandomForestでfit
regressor = RandomForestRegressor(max_features='auto')
regressor.fit(X_train, y_train)

どのfeatureが重要か→day0：feature_39,43,37,5,42  

Note:  
悪いモデルにおいて重要性が低いとみなされる(cross-validation scoreが低い)features  
→良いモデルにとってとても重要となる可能性がある  

重要度を計算する前に、held-out(保持された)setを用いて  
(もしくはcross-validationを持ちいるとより良く)モデルの予測力を評価することが大切    
  
Premmutation importance はそれ自体は内在的なfeatureの予測力を反映しないが、  
あるモデルにおいてそのfeatureがどれほど重要かを反映している  
featureの重要性の本格的な研究が（また、たくさんのCPUを用いるのが）重要なのは言うまでもない  
"Feature selection using the Boruta-SHAP package".
しかし、全体のfeatureの重要性のランキングは全体の物語をつづるわけではない   
"TabNet and interpretability: Jane Street example"   
それぞれ全ての計算においてどのfeaturesが重要かを示し、  
静的な全体のランキングが示すよりも過程がダイナミックである


## 14,Is there any correlation between day 100 and day 200?
各日がindependent(独立)かどうか??  

→actionとrespの単純な相関は0.54(※単一でrespへの強い相関を示すfeatureはない)  
Tag28:feature120～128  
→相関が大きいfeatureを減らせる(60代のfeatureから始めればよい)


In [ ]:
#day100＆day200の比較
#（時間的に離れていて、一時的な漏れ(leakage)を減らせる）  
##Pearson pairwise correlation matrix(ピアソンのペアワイズ相関行列)
#すごい大きな

##diliverging colormapを用いる
#赤はプラスの線形の相関を示し、青は線形の非相関を示す

#day_100と200のcolumnを作り結合
day_100 = train_data.loc[train_data['date'] == 100]
day_200 = train_data.loc[train_data['date'] == 200]
day_100_and_200 = pd.concat([day_100, day_200])
#corr:相関係数を計算(pearsonの方法で)
#.style.background_gradient:ヒートマップっぽくする
#set_precision:表示桁数制限(小数点以下第2位まで)
day_100_and_200.corr(method='pearson').style.background_gradient(cmap='coolwarm', axis=None).set_precision(2)


* actionの単純な定義とrespの値の間にはたった0.54の相関しか見られない  
* また、一つでrespに強い相関を持つようなfeatureは存在しない　　

Spearman's rank correlation coefficient  
Spearmanの順位相関係数  
→financial dataにおいてはより**apropo**（適切？）である  
それは単純に、そのような相関を見るために、pandas.DataFrame.corrの中の、method='pearson'をmethod='spearman'に変更することでできる  
  
### Tag28 section
そこの周りをnavigateすることは、よりいくつかのかなりcurious regionsがあるように見える    
例えば120から129の間のfeaturesをplotすると以下のようになる

In [ ]:
subset = day_100_and_200[["feature_120", "feature_121", "feature_122", "feature_123", "feature_124", "feature_125", "feature_126", "feature_127", "feature_128", "feature_129"]]
subset.corr(method='pearson').style.background_gradient(cmap='coolwarm', low=1, high=0, axis=None).set_precision(2)


Tag 28 features
* feature_120&121：Tag4(resp_1)
* feature_122&123：Tag0(resp_4)
* feature_124&125：Tag3(resp_2)
* feature_126&127：Tag2(resp_3)
* feature_128&129：Tag1(resp)
つまり、
* resp & resp_4の線形相関：0.98
* resp & resp_4の線形相関：0.97
* resp & resp_4の線形相関：0.94
* resp & resp_4の線形相関：0.89

### High correlations



In [ ]:
#相関が|0.992|以上のfeaturesのpairをplot

features_day_100 = day_100.iloc[:, 7:137]
features_day_200 = day_100.iloc[:, 7:137]
features_100_and_200 = pd.concat([features_day_100, features_day_200])

# code from: https://izziswift.com/list-highest-correlation-pairs-from-a-large-correlation-matrix-in-pandas/

#一定の値以上の相関係数をピックアップ
def corrFilter(x: pd.DataFrame, bound:float):
    xCorr = x.corr()
    xFiltered = xCorr[((xCorr >= bound) | (xCorr <= -bound)) & (xCorr !=1.000)]
    xFlattened = xFiltered.unstack().sort_values().drop_duplicates()
    return xFlattened
corrFilter(features_100_and_200, .992).to_frame()

我々のモデルにおいて、最終的なfeaturesの数は確実に削減できるようだ  
始めるのに良い場所は、60台のfeatures(Tag22)を見ること  
(それらの間にはたくさんの相関があったたため)   


## 15,The test data
総経過時間wall time≒3.5h　→　提出前のスクリプトのテストが重要(いつもそうだけど)  
より小さいtest data file(36MB);example_test.csvが与えられている  
これは15000越えのrowsを持ち、3日間のデータを示す  
これはtrain.csvの中の130のfeaturesと各取引のweightからなる  
ちなみにexample_test.csvはrespデータは含まない  


3daysというのはday0,1,2を表し、day2は一日の終わりの方のデータのみしかない
(train.csv fileと同様)　→扱いに注意



## 16,Evaluation：utility score

このコンペはutility scoreによって評価される  
このスコアのより詳細な説明と計算は下のnotebookを参照  
"Understanding the Utility Score Function" written by Renata Ghisloti Duarte de Souza  
"Utility Function and Patterns in Missing Values" written by Leonie  
"Jane Street: Super Fast Utility Score Function" written by Yirun Zhang  

'off-line' evaluation  
Found the Holy Grail: GroupTimeSeriesSplit" written by Jorijn Jacko Smit
"Purged Rolling Time Series CV Split" written by marketneutral



### PostScript(追記)  
自分自身でML戦略を開発したいならば、oddsはあなたに対して積み重ねられる?  
1つの真の投資戦略を作ることは、100を作成するくらい大変  
複雑さは圧倒的  
Marcos Lopez de Prado in "Advances in Financial Machine Learning")

それは、実際よりも数学的・規則的に見える  
その正確さは明らかだが、不正確さは隠されている  
その荒野は待ち構えている  

If you have any problems or improvements,please comment.  
Thank you for reading.  

made by Japanese rookie momijiro